# Scraping Instructor Information from DrivingKing.hk

**Overview**: This notebook scrapes instructor data from the DrivingKing.hk coaching directory using Selenium. It collects both summary information (from the cards) and detailed profile data (from modal/profile pages), saving it into a structured dictionary.




In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

### Extract Basic Info from Coach Card

In [2]:

def get_card_info(coach_card):
    """
    Gets basic card info of each instructor in a page.
    """
    WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "textRow_text")))
    name = coach_card.find_element(By.CLASS_NAME, "coachCard_name").text
    area = coach_card.find_element(By.CLASS_NAME, "coachCard_district").text
    text_rows = [i.text for i in coach_card.find_elements(By.CLASS_NAME, "textRow_text")]
    try:
        car_model = text_rows[0]
    except:
        car_model = None
        
    try:
        regular_lessons = text_rows[1]
    except:
        regular_lessons = None
    
    try:
        supplementary_lessons = text_rows[2]
    except:
        supplementary_lessons = None

    data = {
        "name": name,
        "area": area,
        "regular lesson fee": regular_lessons,
        "supplementary lesson fee": supplementary_lessons
    }

    return data

### Extract Detailed Info from Profile Page

In [4]:
def safe_find_text(parent, by, value):
    try: 
        return parent.find_element(by, value).text
    except NoSuchElementException:
        return None
    except AttributeError:
        return None

def get_more_details(profile):
    """
    Gets additi0nal info for each instructor after clicking their card.
    """

    try:
        WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "coach_description_content")))
    except TimeoutException:
        print("Element not found, skipping...")
            
    
    try:
        name = profile.find_element(By.CLASS_NAME, "coachName").text
    except:
        name = None
    
    try:
        desc = profile.find_element(By.CLASS_NAME, "coach_description_content").text
    except: 
        desc = None

    try:
        if profile.find_element(By.CLASS_NAME, "coachMaleSex"):
            gender = "M"
    except NoSuchElementException:
        try:
            if profile.find_element(By.CLASS_NAME, "coachFemaleSex"):
                gender = "F"
        except NoSuchElementException:
            gender = None
    
    # for both car option (price_type[0]) and pickup location (price_type[1]), usually
    price_type = profile.find_elements(By.CLASS_NAME, "price_type")

    is_car = False # icon for on car options is present
    is_loc = False
    teach_disabled = False
    for row_text in price_type:
        try: 
            if (row_text.find_element(By.CSS_SELECTOR, ".icon_image.carType")):
                is_car = True
                
        except NoSuchElementException:
            pass
            
        try: 
            if (row_text.find_element(By.CSS_SELECTOR, ".icon_image.place")):
                is_loc = True
        except NoSuchElementException:
            pass

        try: 
            if (row_text.find_element(By.CSS_SELECTOR, ".icon_image.disabled")):
                teach_disabled = True
        except NoSuchElementException:
            pass

    if (is_car):
        car_spec = [i.text for i in price_type[0].find_elements(By.CSS_SELECTOR, "tbody tr")]
        num_cars = len(car_spec)
    
        # for multiple car options
        if num_cars >= 1:
            try:
                car_type = []
                car_model = []
                for i in range(num_cars):
                    temp = car_spec[i].split("\n")
                    car_type.append(temp[0].strip())
                    car_model.append(temp[1].strip())
                car_type = ", ".join(car_type)
                car_model = ", ".join(car_model)

            # Either no car type or no car model
            except:
                car_type = None
                car_model = None
            
        else:
            car_type = None
            car_model = None

        # if car and location icons are present
        if (is_loc):
            pickup_loc = price_type[1].find_element(By.CLASS_NAME, "field_value").text

        # only car icon is present
        else:
            pickup_loc = None
                
                
    # only pickup location icon is present        
    elif ( not is_car ) and ( is_loc ) :
        pickup_loc = price_type[0].find_element(By.CLASS_NAME, "field_value").text
        car_type = None
        car_model = None
        num_cars = 0
   
        
    try:
        awards = profile.find_element(By.CLASS_NAME,"price_type_content.awards").text
    except NoSuchElementException:
        awards = None

    try:
        session_duration = profile.find_element(By.CLASS_NAME, "description").text
    except:
        session_duration = None
        
    # on exam day
    try:
        learn_exam = profile.find_element(By.CLASS_NAME, "priceTable.learnExam")
    except:
        learn_exam = None
    
    try:
        rent_car_type = learn_exam.find_element(By.CLASS_NAME, "row_value.firstCol").text
    except:
        rent_car_type = None
        
    try:
        toll_fee = learn_exam.find_element(By.CLASS_NAME, "row_value.secondCol").text
    except:
        toll_fee = None
    try:
        rent_car_fee = learn_exam.find_element(By.CLASS_NAME, "row_value.thirdCol").text
    except:
        rent_car_fee = None
        
    # discounted pricing 
    try:
        extra_lesson = profile.find_element(By.CLASS_NAME, "priceTable.extraLesson")
    except:
        extra_lesson = None

    try:
        discounted_rent_car_type = extra_lesson.find_element(By.CLASS_NAME, "row_value.firstCol_grid4").text
    except:
        discounted_rent_car_type = None

    try:
        licensed_driver = extra_lesson.find_element(By.CLASS_NAME, "row_value.secondCol_grid4").text
    except:
        licensed_driver = None
    
    try:
        school_card_holders = extra_lesson.find_element(By.CLASS_NAME, "row_value.thirdCol_grid4").text
    except:
        school_card_holders = None
    
    try:
        faculty_members = extra_lesson.find_element(By.CLASS_NAME, "row_value.fourthCol_grid4").text
    except:
        faculty_members = None

    try:
        insurance_cert = profile.find_element(By.CLASS_NAME, "coach_certificateAndInsurance_item").text
    except:
        insurance_cert = None
        
    data = {
      "gender": gender,
      "description": desc,
      "car_type": car_type,
      "car model/s": car_model,
      "pickup location": pickup_loc,
      "disability accomodation": teach_disabled,
      "awards": awards,
      "session duration": session_duration,
      "number of car options": num_cars,
      "rental car type/s": rent_car_type,
      "toll fee": toll_fee,
      "car rental fee": rent_car_fee,
      "rental car type": discounted_rent_car_type,
      "licensed driver fee": licensed_driver,
      "school card holders": school_card_holders,
      "faculty members fee": faculty_members,
      "insurance": insurance_cert
    }

    return data


### Scrape All Pages

In [5]:
driver = webdriver.Chrome()
url = "https://www.drivingking.hk/webCoach/searchCoach?carType=&carModel=&coachName=&courseOrderOperationId=&district=&language=&offset=&operationType=&orderStatus=&orderType=&pickUpLocationId=&pickUpLocation=&schoolPlaceId=&schoolPlace=&pageNo=1&fromPage="
driver.get(url)
current_page = 1
max_page = 11

profile_db = []

while current_page <= max_page:
    WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "coachCard")))
    cards = driver.find_elements(By.CLASS_NAME, "coachCard")
    
    for i in range(len(cards)):
        cards = driver.find_elements(By.CLASS_NAME, "coachCard")
        card = cards[i]
        basic_info = get_card_info(card)

        driver.execute_script("arguments[0].scrollIntoView(true);", card)
        driver.execute_script("arguments[0].click();", card)

        WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "profileContainer")))
        profile = driver.find_element(By.CLASS_NAME, "profileContainer")
        additional_info = get_more_details(profile)
        driver.back()
        WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "coachCard")))
        merged = {**basic_info, **additional_info}
        profile_db.append(merged)

    try:
        next_button = driver.find_element(By.ID, "mobilePagination_nextPage")
    except NoSuchElementException:
        try:
            next_button = driver.find_element(By.ID, "nextPageButton")
            print("page: ", current_page)
        except NoSuchElementException:
            print("Reached the last page.")
            break  
    driver.execute_script("arguments[0].click();", next_button)
    WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "coachCard")))
    
    current_page += 1

page:  2
page:  3
page:  4
page:  5
page:  6
Element not found, skipping...
page:  7
page:  8
page:  9
Reached the last page.


In [6]:
import csv
output_file = "driving_instructors.csv"
with open(output_file, mode="w", encoding="utf-8-sig", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=profile_db[0].keys())
    writer.writeheader()
    writer.writerows(profile_db)
print("success")

success


# Basic Analysis

First re-store the collected data from csv file to a dataframe as an accessible relational database.

In [11]:
with open(output_file, mode="r", encoding="utf-8-sig") as in_file:
    reader = csv.reader(in_file)
    header = next(reader)
    header = [cap_header.capitalize() for cap_header in header]
    print(header)
    reader = list(reader)

    name_index = header.index("Name")
    
    area_index = header.index("Area")
    area_val = [row[area_index] for row in reader]
    
    reg_lesson_fee_index = header.index("Regular lesson fee")
    reg_lesson_fee_value = [row[reg_lesson_fee_index] for row in reader]
    
    supplementary_fee_index = header.index("Supplementary lesson fee")
    supplementary_lesson_fee_value = [row[supplementary_fee_index] for row in reader]
    
    gender_index = header.index("Gender")
    gender_value = [row[gender_index] for row in reader]
    
    description_index = header.index("Description")
    car_type_index = header.index("Car_type")
    car_type_value = [row[car_type_index] for row in reader]
    
    car_model_index = header.index("Car model/s")
    car_model_value = [row[car_model_index] for row in reader]
    
    pickup_loc_index = header.index("Pickup location")
    
    disability_index = header.index("Disability accomodation")
    disability_value = [row[disability_index] for row in reader]
    
    awards_index = header.index("Awards")
    awards_value = [row[awards_index] for row in reader]
    
    session_index = header.index("Session duration")
    session_value = [row[session_index] for row in reader]
    
    num_cars_index = header.index("Number of car options")
    num_cars_value = [row[num_cars_index] for row in reader]
    
    rent_car_type_index= header.index("Rental car type/s")
    rent_car_type_value = [row[rent_car_type_index] for row in reader]
    
    toll_fee_index = header.index("Toll fee")
    toll_fee_value = [row[toll_fee_index] for row in reader]
    
    car_rental_fee_index = header.index("Car rental fee")
    car_rental_fee_value = [row[car_rental_fee_index] for row in reader]
    
    car_rental_type_index = header.index("Rental car type")
    car_rental_type_value = [row[car_rental_type_index] for row in reader]
    
    licensed_fee_index = header.index("Licensed driver fee")
    licensed_fee_value = [row[licensed_fee_index] for row in reader]
    
    school_card_holders_index = header.index("School card holders")
    school_card_holders_value = [row[school_card_holders_index] for row in reader]
    
    
    faculty_fee_index = header.index("Faculty members fee")
    faculty_fee_value = [row[faculty_fee_index] for row in reader]
    
    insurance_index = header.index("Insurance")
    insurance_value = [row[insurance_index] for row in reader]

    

['Name', 'Area', 'Regular lesson fee', 'Supplementary lesson fee', 'Gender', 'Description', 'Car_type', 'Car model/s', 'Pickup location', 'Disability accomodation', 'Awards', 'Session duration', 'Number of car options', 'Rental car type/s', 'Toll fee', 'Car rental fee', 'Rental car type', 'Licensed driver fee', 'School card holders', 'Faculty members fee', 'Insurance']


Next, for each field we count the number of unique categories and number of instances per category. We use the code blocks above and perform syntactic modifications to turn them into runnable code blocks for our purpose above

Of course, we can make a simpler function, but the code below explains the logic more clearly.

In [12]:
def pretty_print(category):
    for key, value in category.items():
        if key == '':
            print(f"No entry: {value}")
        else: 
            print(f"{key}: {value}")

In [13]:
string = """area_index = header.index("Area")
    area_val = [row[area_index] for row in reader]
    reg_lesson_fee_index = header.index("Regular Lesson Fee")
    reg_lesson_fee_value = [row[reg_lesson_fee_index] for row in reader]
    supplementary_fee_index = header.index("Supplementary Lesson Fee")
    supplementary_lesson_fee_value = [row[supplementary_lesson_fee_index] for row in reader]
    gender_index = header.index("Gender")
    gender_value = [row[gender_index] for row in reader]
    car_type_index = header.index("Car Type/s")
    car_type_value = [row[car_type_index] for row in reader]
    car_model_index = header.index("Car Model/s")
    car_model_value = [row[car_model_index] for row in reader]
    disability_index = header.index("Disability Accomodation")
    disability_value = [row[disability_index] for row in reader]
    awards_index = header.index("Awards")
    awards_value = [row[awards_index] for row in reader]
    session_index = header.index("Session Duration")
    session_value = [row[session_index] for row in reader]
    num_cars_index = header.index("Number of Car Options")
    num_cars_value = [row[num_cars_index] for row in reader]
    rent_car_type_index= header.index("Rental Car Type/s")
    rent_car_type_value = [row[rent_car_type_index] for row in reader]
    toll_fee_index = header.index("Toll Fee")
    toll_fee_value = [row[toll_fee_index] for row in reader]
    car_rental_fee_index = header.index("Car Rental Fee")
    car_rental_fee_value = [row[car_rental_fee_index] for row in reader]
    car_rental_type_index = header.index("Rental Car Type")
    car_rental_type_value = [row[car_rental_type_index] for row in reader]
    licensed_fee_index = header.index("Licensed Driver Fee")
    licensed_fee_value = [row[licensed_fee_index] for row in reader]
    school_card_holders_index = header.index("School Card Holders")
    school_card_holders_value = [row[school_card_holders_index] for row in reader]
    faculty_fee_index = header.index("Faculty Members Fee")
    faculty_fee_value = [row[faculty_fee_index] for row in reader]
    insurance_index = header.index("Insurance")
    insurance_value = [row[insurance_fee_index] for row in reader]
    """
my_list = []
add_category = True
for i in string.split("\n"):
    # print(counter)
    first_word = i.strip().split(" ")[0]
    category = first_word.split("_")[0]
    if (add_category):
        my_list.append(category)
        add_category = False

    else:
        add_category = True
    my_list.append(first_word)
        
for i in range(0, len(my_list), 3):
    if i + 2 >= len(my_list):
        break  # avoid index out of range
    category = my_list[i]
    index = my_list[i + 1]
    value = my_list[i + 2]

# uncomment code below to print copy-pasteable code for next section
#     print(f"""
# {category}_category = set({value})
# {category}_count = {{}}
# for i in {category}_category:
#     {category}_count[i] = {value}.count(i)

# print({category}_count)
# """)

Now, we perform basic counting statistics.

In [16]:
area_category = set(area_val)
area_count = {}
for i in area_category:
    area_count[i] = area_val.count(i)

reg_category = set(reg_lesson_fee_value)
reg_count = {}
for i in reg_category:
    reg_count[i] = reg_lesson_fee_value.count(i)

supplementary_category = set(supplementary_lesson_fee_value)
supplementary_count = {}
for i in supplementary_category:
    supplementary_count[i] = supplementary_lesson_fee_value.count(i)

gender_category = set(gender_value)
gender_count = {}
for i in gender_category:
    gender_count[i] = gender_value.count(i)

car_type_category = set(car_type_value)
car_type_count = {}
for i in car_type_category:
    car_type_count[i] = car_type_value.count(i)

car_model_category = set(car_model_value)
car_model_count = {}
for i in car_model_category:
    car_model_count[i] = car_model_value.count(i)

disability_category = set(disability_value)
disability_count = {}
for i in disability_category:
    disability_count[i] = disability_value.count(i)

awards_category = set(awards_value)
awards_count = {}
for i in awards_category:
    awards_count[i] = awards_value.count(i)

session_category = set(session_value)
session_count = {}
for i in session_category:
    session_count[i] = session_value.count(i)

num_category = set(num_cars_value)
num_count = {}
for i in num_category:
    num_count[i] = num_cars_value.count(i)

rent_car_type_category = set(rent_car_type_value)
rent_car_type_count = {}
for i in rent_car_type_category:
    rent_car_type_count[i] = rent_car_type_value.count(i)

toll_category = set(toll_fee_value)
toll_count = {}
for i in toll_category:
    toll_count[i] = toll_fee_value.count(i)

car_rental_fee_category = set(car_rental_fee_value)
car_rental_fee_count = {}
for i in car_rental_fee_category:
    car_rental_fee_count[i] = car_rental_fee_value.count(i)

car_rental_type_category = set(car_rental_type_value)
car_rental_type_count = {}
for i in car_rental_type_category:
    car_rental_type_count[i] = car_rental_type_value.count(i)

licensed_category = set(licensed_fee_value)
licensed_count = {}
for i in licensed_category:
    licensed_count[i] = licensed_fee_value.count(i)

school_category = set(school_card_holders_value)
school_count = {}
for i in school_category:
    school_count[i] = school_card_holders_value.count(i)

faculty_category = set(faculty_fee_value)
faculty_count = {}
for i in faculty_category:
    faculty_count[i] = faculty_fee_value.count(i)

insurance_category = set(insurance_value)
insurance_count = {}
for i in insurance_category:
    insurance_count[i] = insurance_value.count(i)


<br>

### **Parameter: `area`**

**Description:**  
The `area` parameter represents the geographical region associated with each data entry. It includes various combinations of three main regions in Hong Kong:  
- **香港** (Hong Kong Island)  
- **九龍** (Kowloon)  
- **新界** (New Territories)

---

### **Data Summary**

| Area              | Count |
|-------------------|-------|
| 香港              | 25    |
| 香港 九龍 新界    | 18    |
| 香港 九龍         | 4     |
| 九龍              | 14    |
| 九龍 新界         | 126   |
| 新界              | 4     |

---

### **Analysis**

- The most frequent region combination is **九龍 新界** with **126 entries**, indicating a significant concentration in these areas.
- **香港** (25) and **九龍** (14) also have moderate representation individually.
- Combinations involving all three regions (**香港 九龍 新界**) or **香港 九龍** are less common.
- **新界** alone has minimal representation (**4 entries**), suggesting it is often grouped with 九龍.


In [106]:
pretty_print(area_count)

香港: 25
香港 九龍 新界: 18
香港 九龍: 4
九龍: 14
九龍 新界: 126
新界: 4


<br>

### Parameter: `regular lesson fees`

**Description:**  
This parameter represents the stated cost per session for a regular driving lesson. It includes:
- Exact prices (e.g., `$330 per session`)
- Price ranges (e.g., `$330–350 per session`)
- Missing entries labeled as `No entry`

---

### Data Summary
```
| Fee Range            | Count |
|----------------------|-------|
| No entry             | 18    |
| $300 per session     | 7     |
| $310 per session     | 4     |
| $320 per session     | 13    |
| $330 per session     | 27    |
| $330–350 per session | 1     |
| $330–450 per session | 1     |
| $340 per session     | 23    |
| $340–350 per session | 1     |
| $340–360 per session | 1     |
| $340–370 per session | 1     |
| $350 per session     | 59    |
| $350–380 per session | 1     |
| $350–420 per session | 1     |
| $355 per session     | 1     |
| $360 per session     | 5     |
| $370 per session     | 2     |
| $380 per session     | 13    |
| $380–450 per session | 1     |
| $380–500 per session | 1     |
| $400 per session     | 6     |
| $420 per session     | 3     |
| $450 per session     | 1     |
```
---

**Analysis**

- The most common fee is **HKD 350 per session**, appearing 59 times, suggesting it is the standard market rate.
- **HKD 330 per session** (27 entries) and **$HKD 40 per session** (23 entries) are also frequent, representing the mid-range.
- A few entries use **price ranges**, possibly indicating negotiable or flexible rates.
- Higher-end prices such as **HKD 400 to HKD 450 per session** are rare.
- 18 records have no fee listed, possibly due to incomplete data or private arrangements.


In [111]:
pretty_print(reg_count)

No entry: 18
$320/節: 13
$380-450/節: 1
$330/節: 27
$330-350/節: 1
$350/節: 59
$370/節: 2
$350-420/節: 1
$380-500/節: 1
$355/節: 1
$350-380/節: 1
$310/節: 4
$300/節: 7
$330-450/節: 1
$420/節: 3
$340-360/節: 1
$360/節: 5
$340/節: 23
$380/節: 13
$340-370/節: 1
$400/節: 6
$450/節: 1
$340-350/節: 1


<br>

### Parameter: `supplementary lesson fees`

**Description:**  
This parameter reflects the cost per session for supplementary (additional) driving lessons. These lessons are usually optional or tailored sessions beyond the standard curriculum. Prices are either fixed or presented as ranges, and some entries may be missing.

**Analysis**

- **HKD 450 per session** is the most frequently listed rate, suggesting it is the prevailing rate for supplementary lessons.
- **HKD 400** and **HKD 500 per session** also appear frequently, indicating a broad standard range between $400–$500.
- The presence of many ranged fees shows that supplementary lesson pricing is often flexible or negotiable.


In [112]:
pretty_print(supplementary_count)

No entry: 24
$425-475/節: 1
$380-450/節: 1
$425/節: 1
$480/節: 12
$400-500/節: 4
$420-460/節: 1
$430-500/節: 1
$350/節: 2
$440-450/節: 1
$400-450/節: 10
$370/節: 1
$380-420/節: 1
$450-600/節: 1
$520-650/節: 1
$400-570/節: 1
$500/節: 14
$700/節: 1
$350-450/節: 1
$450-500/節: 13
$400-550/節: 1
$440/節: 1
$430-450/節: 2
$430-460/節: 1
$550-650/節: 1
$450-490/節: 1
$350-400/節: 4
$420-480/節: 1
$430/節: 3
$420-550/節: 1
$425-500/節: 1
$425-450/節: 1
$350-425/節: 1
$450-530/節: 1
$500-600/節: 2
$375-450/節: 1
$420-470/節: 1
$420/節: 6
$550/節: 4
$370-420/節: 1
$430-480/節: 1
$500-550/節: 3
$380-550/節: 1
$400-430/節: 1
$450-550/節: 2
$400/節: 21
$450/節: 31
$420-450/節: 1
$400-420/節: 3


<br>

### Parameter: `gender`

**Description:**  
This parameter represents the gender of the driving instructors listed. It’s typically self-declared or inferred from profiles.

---

**Top Gender Distribution**

| Gender | Count |
|--------|-------|
| Male   | 188   |
| Female | 3     |

---

**Analysis**

- The instructor pool is **overwhelmingly male** (188 out of 191), indicating a strong gender imbalance in the industry.
- Female representation is minimal, making up less than **2%** of the total entries.


In [113]:
pretty_print(gender_count)

F: 3
M: 188


<br>

### Parameter: `car type`

**Description:**  
This parameter indicates the types of vehicles the instructor is qualified or available to teach with. It includes private cars (automatic/manual) and light goods vehicles (automatic/manual), sometimes in combination.

---

**Top 3 Most Common Car Types**

| Car Type                                         | Count |
|--------------------------------------------------|-------|
| Private Car - Automatic                          | 75    |
| No entry                                         | 57    |
| Private Car - Automatic, Light Goods - Manual    | 19    |

---

**Analysis**

- The most common vehicle type is **Private Car - Automatic**, with 75 instructors offering it.
- Many instructors provide combinations, indicating versatility across transmission types and vehicle classes.
- A significant number (57) did not list their vehicle type, possibly due to missing data or profile omission.


In [124]:
pretty_print(car_type_count)

No entry: 57
輕型貨車-自動波: 11
私家車-自動波: 75
輕型貨車-自動波, 輕型貨車-棍波: 4
私家車-自動波, 輕型貨車-自動波, 輕型貨車-棍波: 3
輕型貨車-棍波: 15
私家車-自動波, 輕型貨車-自動波: 4
私家車-自動波, 私家車-棍波, 輕型貨車-自動波: 1
私家車-自動波, 輕型貨車-棍波: 19
私家車-自動波, 私家車-棍波: 2


<br>

### Parameter: `car model`

**Description:**  
This parameter lists the specific car models used by driving instructors. Entries include both individual vehicles and combinations (when instructors use multiple models). Models span common private cars and vans such as the Toyota Hiace.

---

**Top 3 Most Common Car Models**

| Car Model         | Count |
|-------------------|-------|
| No entry          | 57    |
| Toyota Prius C    | 38    |
| Toyota Hiace      | 26    |

---

**Analysis**

- **Toyota Prius C** is the most commonly listed individual model, suggesting it is popular among instructors for its efficiency and compact size.
- **Toyota Hiace**, a light goods vehicle, appears frequently, indicating that many instructors also train for commercial or transport vehicle licenses.
- A significant portion of entries have **no listed model**, possibly due to incomplete data or intentionally omitted fields.


In [125]:
pretty_print(car_model_count)

No entry: 57
Toyota Prius C: 38
Honda Jazz, Toyota Yaris: 1
Toyota Prius C, Toyota Hiace: 12
Honda Jazz, Toyota Hiace: 3
Toyota Sienta, Toyota Hiace: 1
Honda Jazz, Toyota Echo, Toyota Hiace: 1
Toyota Yaris: 10
Toyota Hiace 200, Toyota Hiace: 1
Toyota Hiace, Toyota Hiace: 3
Honda Fit Hybrid, Toyota Hiace: 1
Toyota Corolla, Toyota Hiace: 1
Honda Jazz RS 2024: 1
Toyota Yaris, Toyota Hiace: 1
Toyota Prius C, Toyota Echo: 1
Toyota Prius C, Toyota Hiace, Toyota Hiace: 3
BYD Dolphin, Toyota Hiace: 1
Toyota Spade: 1
Honda Fit Hybrid: 1
Toyota Aqua, Toyota Hiace: 1
Kia Morning, Toyota Hiace: 2
Toyota Echo: 1
Nissan e-power: 2
Nissan Note: 1
Honda Jazz: 17
BYD Dolphin: 3
Toyota Hiace: 26


<br>

### Parameter: `disability accommodation`

**Description:**  
This parameter indicates whether the driving instructor offers accommodations or services tailored for individuals with disabilities.

---

**Top Values**

| Accommodation Available | Count |
|--------------------------|-------|
| No (False)               | 174   |
| Yes (True)               | 17    |

---

**Analysis**

- Only **17 instructors** out of 191 explicitly offer disability accommodations, making up less than **9%** of the total.
- The vast majority **(91%)** do not provide or have not indicated any accommodations, highlighting a potential gap in accessibility within the driving instruction sector.


In [126]:
pretty_print(disability_count)

FALSE: 174
TRUE: 17


<br>

### Parameter: `awards`

**Description:**  
This parameter lists any honors, awards, certifications, or professional memberships that instructors have received. These recognitions may indicate quality, experience, or industry acknowledgment.


In [127]:
pretty_print(awards_count)

No entry: 187
運輸署優質駕駛導師
香港汽車高級駕駛協會會員
查看證書: 2
全港公開安全駕駛比賽
查看證書: 1
全港公開安全駕駛比賽優異獎
2001年全港公開組私家車自動波第二名
Driver of The Year 1993第一名
查看證書: 1


<br>

### Parameter: `session duration (45 minutes/session HKD)`

**Description:**  
This parameter standardizes the cost of a driving lesson based on a 45-minute session. The value **191** indicates the total number of instructors with session durations recorded in this format.

- All **191 instructors** use a 45-minute session structure, suggesting it is the industry standard.

In [128]:
pretty_print(session_count)

（45分鐘/堂 HKD）: 191


<br>

### Parameter: `number of car options offered`

**Description:**  
This parameter represents how many different car models or types each instructor offers for driving lessons. It reflects the flexibility or variety available in their teaching fleet.

---

**Top 3 Most Common Counts**

| Number of Car Options | Count |
|------------------------|-------|
| 1                      | 101   |
| 0 (No info)            | 52    |
| 2                      | 33    |

---

**Analysis**

- A **majority (101 instructors)** offer just **one car option**, likely aligning with the most common license type (e.g., private automatic).
- **52 instructors** did not specify any car options, indicating missing data or unlisted details.
- A smaller group offers **2 or more vehicle types**, showing greater versatility or broader service coverage.


In [129]:
pretty_print(num_count)

0: 52
3: 4
1: 101
2: 33
7: 1


<br>

### Parameter: `rent car types`

**Description:**  
This parameter details the types of rental cars available for driving lessons, including distinctions between vehicle class (light goods vehicle vs. private car) and transmission type (manual vs. automatic).

---

**Top 3 Most Common Rent Car Types**

| Rent Car Type          | Count |
|-----------------------|-------|
| Private Car - Automatic| 125   |
| Light Goods Vehicle - Automatic | 29    |
| Light Goods Vehicle - Manual  | 19    |

---

**Analysis**

- The most commonly offered rental car is a **private car with automatic transmission**, reflecting demand and learner preference.
- Light goods vehicles, both automatic and manual, are also available but less frequently.
- **18 entries have no specified rent car type**, indicating incomplete data or unlisted options.


In [130]:
pretty_print(rent_car_type_count)

No entry: 18
輕型貨車 - 棍波: 19
輕型貨車 - 自動波: 29
私家車 - 自動波: 125


<br>

### Parameter: `toll fee`

**Description:**  
This parameter represents the toll fee (in HKD) associated with the driving lessons or vehicles used. The values indicate the amount charged or recorded per toll usage.

---

**Top 3 Most Common Toll Fees**

| Toll Fee (HKD) | Count |
|----------------|-------|
| 350            | 60    |
| 330            | 29    |
| 340            | 26    |

---

**Analysis**

- The most frequent toll fee charged is **350 HKD**, followed by **330 HKD** and **340 HKD**, indicating a typical range around these amounts.
- A notable number of entries (**18**) have no recorded toll fee, possibly due to omission or zero charges.
- Fees vary moderately, with some lower (300 HKD) and higher (450 HKD) outliers.


In [131]:
pretty_print(toll_count)

310: 4
No entry: 18
330: 29
450: 1
420: 3
340: 26
400: 6
320: 13
355: 1
370: 2
360: 5
300: 7
350: 60
380: 16


<br>

### Parameter: `car rental fee`

**Description:**  
This parameter shows the rental fee amount (in HKD) charged for car rentals used is **HKD 2,300** during driving lessons.

- This consistency suggests a standardized rental pricing across most instructors.

In [134]:
pretty_print(car_rental_fee_count)

No entry: 18
2300: 173


<br>

### Parameter: `car rental fee type`

**Description:**  
This parameter specifies the type of vehicle for which the car rental fee applies, categorized by vehicle class and transmission type.

---

**Top 3 Most Common Car Rental Fee Types**

| Rental Fee Type           | Count |
|--------------------------|-------|
| Private Car - Automatic   | 137   |
| Light Goods Vehicle - Automatic | 28    |
| Light Goods Vehicle - Manual  | 19    |

---

**Analysis**

- The majority of rental fees are associated with **private cars with automatic transmission**.
- Light goods vehicles, both automatic and manual, make up the next largest groups.
- Few entries are missing or represent less common categories such as private cars with manual transmission.


In [135]:
pretty_print(car_rental_type_count)

No entry: 6
私家車 - 自動波: 137
輕型貨車 - 棍波: 19
輕型貨車 - 自動波: 28
私家車 - 棍波: 1


<br>

### Parameter: `fee for licensed drivers (renewals)`

**Description:**  
This parameter indicates the discounted lesson fee offered to drivers who are renewing their licenses, reflecting possible price reductions for experienced drivers.

---

**Top 3 Most Common Fees**

| Discounted Fee (HKD) | Count |
|----------------------|-------|
| 450                  | 46    |
| 400                  | 32    |
| 500                  | 32    |

---

**Analysis**

- The most common discounted fees are **450 HKD**, **400 HKD**, and **500 HKD**, suggesting these are typical renewal rates.
- A notable number of entries have **no entry (6)** or explicitly marked as "none" (18), indicating no discount or missing data.
- Other fee values vary widely, showing some flexibility or differing pricing strategies among instructors.


In [136]:
pretty_print(licensed_count)

No entry: 6
530: 1
400: 32
375: 1
490: 1
650: 1
390: 1
475: 1
550: 11
350: 4
430: 6
460: 3
450: 46
340: 1
無: 18
470: 1
480: 14
420: 8
500: 32
425: 1
380: 2


<br>

### Parameter: `school fee`

**Description:**  
This parameter represents the fees charged to students attending driving lessons at schools, possibly reflecting special rates or packages.

---

**Top 3 Most Common School Fees**

| School Fee (HKD) | Count |
|------------------|-------|
| 450              | 44    |
| 400              | 42    |
| 500              | 20    |

---

**Analysis**

- The most frequent school fees are **450 HKD**, **400 HKD**, and **500 HKD**, indicating these are standard rates for school-based lessons.
- Some entries have **no entry (6)** or are marked as "none" (19), suggesting either missing data or no special school fee applied.
- Other fees vary widely, showing some diversity in pricing structures among different schools or instructors.


In [137]:
pretty_print(school_count)

No entry: 6
430: 8
520: 1
450: 44
420: 10
480: 12
500: 20
400: 42
無: 19
440: 2
550: 8
375: 1
370: 1
425: 4
390: 1
470: 1
350: 9
380: 2


<br>

### Parameter: `faculty fee`

**Description:**  
This parameter refers to special fees or discounts offered to faculty members, possibly reflecting negotiated rates for educational staff.

---

**Top 3 Most Common Faculty Fees**

| Faculty Fee (HKD) | Count |
|-------------------|-------|
| 450               | 62    |
| 400               | 28    |
| 500               | 24    |

---

**Analysis**

- The majority of faculty fees cluster around **450 HKD**, **400 HKD**, and **500 HKD**, indicating common pricing tiers for faculty discounts.
- There are some entries with **no entry (6)** or marked as "none" (18), indicating missing data or no special faculty fee applied.
- A few outliers exist with higher fees such as **600 HKD** and **700 HKD**, showing some variation in discount policies.


In [138]:
pretty_print(faculty_count)

No entry: 6
430: 5
475: 1
450: 62
420: 15
480: 12
500: 24
400: 28
440: 3
550: 5
600: 2
370: 1
無: 18
390: 1
425: 2
350: 5
700: 1


<br>

### Parameter: `insurance certificate`

**Description:**  
This parameter describes the types of insurance coverage provided by instructors for driving lessons, including liability and comprehensive insurance, as well as notes on accident compensation responsibility.

---

**Top 3 Most Common Insurance Types / Status**

| Insurance / Status                              | Count |
|------------------------------------------------|-------|
| Third Party Insurance with half compensation   | 106   |
| Comprehensive Insurance with half compensation | 54    |
| No accident compensation method provided       | 21    |

---

**Analysis**

- Most instructors provide **third party insurance** with the instructor bearing **half of the accident compensation cost** during lessons.
- A significant portion offers **comprehensive insurance** with similar half compensation responsibility.
- There are also multiple entries indicating **no accident compensation method provided** or missing insurance information, which may be a concern for learners.
- A few instructors cover **full compensation** during lessons, but these are less common.


In [139]:
pretty_print(insurance_count)

第三者保險
汽車綜合保險
師傅暫未提供事故賠償方式: 1
第三者保險
學車期間墊底費師傅承擔 全部: 2
第三者保險
師傅暫未提供事故賠償方式: 21
師傅暫未提供保險資訊: 1
師傅暫未提供保險資訊
學車期間墊底費師傅承擔 一半: 2
第三者保險
學車期間墊底費師傅承擔 一半: 106
汽車綜合保險
師傅暫未提供事故賠償方式: 3
第三者保險
汽車綜合保險
學車期間墊底費師傅承擔 一半: 1
汽車綜合保險
學車期間墊底費師傅承擔 一半: 54
